<a href="https://colab.research.google.com/github/averyrair/ChessBAKEN/blob/main/MoveSelector/MoveSelectorBAKEN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
!pip install chess
!pip install stockfish
!apt install stockfish

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
stockfish is already the newest version (14.1-1).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


In [35]:
#@title Import Directories

from google.colab import drive
drive.mount('/content/drive', force_remount=1)
chess_dir = '/content/drive/Shareddrives/Chess Bot BAKEN'

Mounted at /content/drive


In [36]:
#@title Import Libraries

import os
import chess
import torch
import torch.nn as nn
from torch.utils.data import Dataset
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [37]:
#@title Import PGN and Export CSV

if not os.path.exists(os.path.join(chess_dir, 'moveSelectorDataset.csv')):
  pgn_dir = os.path.join(chess_dir, 'KingBase2019-pgn')
  pgnFiles = [os.path.join(pgn_dir, f) for f in os.listdir(pgn_dir) if f.endswith('.pgn')]
  #print(pgnFiles)


In [38]:
#@title Define Fully-Connected Neural Network

class myFCN(nn.Module):
  def __init__(self, inSize, hiddenSizes, outSize):
    super().__init__()
    self.inSize = inSize
    self.hiddenSize = hiddenSizes
    self.outSize = outSize

    self.lin1 = nn.Linear(inSize, hiddenSizes[0])
    self.lin2 = nn.Linear(hiddenSizes[0], hiddenSizes[1])
    self.lin3 = nn.Linear(hiddenSizes[1], outSize)

    self.bn1 = nn.BatchNorm2d(hiddenSizes[0])
    self.bn2 = nn.BatchNorm2d(hiddenSizes[1])

    self.relu = nn.ReLU()

  def forward(self, x):
    x = self.relu(self.bn1(self.lin1(x)))
    x = self.relu(self.bn2(self.lin2(x)))
    x = self.lin3(x)
    return x

In [39]:
#@title Initialize Fully-Connected Neural Network

moveSelector = myFCN(inSize=2560, hiddenSizes=[2200, 2200], outSize=1792)
moveSelector

myFCN(
  (lin1): Linear(in_features=2560, out_features=2200, bias=True)
  (lin2): Linear(in_features=2200, out_features=2200, bias=True)
  (lin3): Linear(in_features=2200, out_features=1792, bias=True)
  (bn1): BatchNorm2d(2200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm2d(2200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
)

In [40]:
#@title Define Dataset

class ChessDataset(Dataset):
    def __init__(self, chessgame_file, chess_dir, transform=None, target_transform=None):
        self.game_labels = pd.read_csv(chessgame_file)
        self.chess_dir = chess_dir  # This is not used. Everything is in the chessgame file.
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.game_labels)

    def __getitem__(self, idx):
        gameState = self.game_labels.iloc[idx, 0]
        label = self.game_labels.iloc[idx, 1]
        if self.transform:
            gameState = self.transform(gameState)
        if self.target_transform:
            label = self.target_transform(label)
        return gameState, label